In [ ]:
!pip install deap
!pip install mealpy
!pip install --upgrade mealpy #upgrade mealpy to the latest version

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten
from deap import base, creator, tools, algorithms
import random
import numpy as np
from mealpy.swarm_based import PSO
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Veri setini yükleyin
dataset_path = '/content/drive/My Drive/DATASET/final_hate_speech.xlsx'
data = pd.read_excel(dataset_path)

In [ ]:
# Metin ve etiket sütunlarını belirleyin
X = data['tweet']
y = data['etiket']

# Etiketleri encode edin
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# TF-IDF Dönüşümü
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(X).toarray()

# Eğitim ve test verisine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def build_model(learning_rate, num_hidden_layers, units_per_layer, activation, dropout_rate, optimizer_name):
    model = Sequential()
    model.add(Dense(units_per_layer, activation=activation, input_shape=(X_train.shape[1],)))
    for _ in range(num_hidden_layers - 1):
        model.add(Dense(units_per_layer, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = {
        "SGD": tf.keras.optimizers.SGD(learning_rate=learning_rate),
        "Adam": tf.keras.optimizers.Adam(learning_rate=learning_rate),
        "RMSProp": tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
    }[optimizer_name]

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def evaluate_ga(individual):
    # GA bireyindeki tüm parametreleri unpack et
    learning_rate, batch_size, num_hidden_layers, units_per_layer, dropout_rate, activation_index, optimizer_index = individual

    # Parametre indekslerini isimlere dönüştür
    activation_functions = ['relu', 'tanh', 'sigmoid']
    optimizers = ['SGD', 'Adam', 'RMSProp']

    activation = activation_functions[int(activation_index)]
    optimizer = optimizers[int(optimizer_index)]

    # Modeli oluştur
    model = build_model(learning_rate=float(learning_rate),
                        num_hidden_layers=int(num_hidden_layers),
                        units_per_layer=int(units_per_layer),
                        activation=activation,
                        dropout_rate=float(dropout_rate),
                        optimizer_name=optimizer)

    # Modeli eğit
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=5, verbose=0)

    # Test doğruluğunu al
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy,

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Birey sınıfı: FitnessMax'i kullanır
creator.create("Individual", list, fitness=creator.FitnessMax)

# DEAP Ayarları
toolbox = base.Toolbox()

toolbox.register("attr_lr", random.uniform, 0.0001, 0.1)  # Öğrenme oranı
toolbox.register("attr_batch_size", random.choice, [16, 32, 64, 128, 256])  # Batch size
toolbox.register("attr_conv_layers", random.randint, 1, 5)  # Convolutional katman sayısı
toolbox.register("attr_num_filters", random.choice, [8, 16, 32, 64, 128])  # Filtre sayısı
toolbox.register("attr_kernel_size", random.choice, [3, 5])  # Kernel boyutu
toolbox.register("attr_activation", random.randint, 0, 2)  # Aktivasyon fonksiyonu
toolbox.register("attr_pooling", random.randint, 0, 1)  # Pooling tipi
toolbox.register("attr_optimizer", random.randint, 0, 2)  # Optimizasyon algoritması
toolbox.register("attr_hidden_layers", random.randint, 1, 5)  # Gizli katman sayısı
toolbox.register("attr_units", random.choice, [64, 128, 256, 512])  # Nöron sayısı
toolbox.register("attr_dropout", random.uniform, 0.1, 0.5)  # Dropout oranı
toolbox.register("mate", tools.cxTwoPoint)  # Çaprazlama fonksiyonu ekleyin
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Mutasyon fonksiyonu ekleyin
toolbox.register("select", tools.selTournament, tournsize=3)  # Seçim fonksiyonu ekleyin
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lr, toolbox.attr_batch_size, toolbox.attr_hidden_layers,
                  toolbox.attr_units, toolbox.attr_dropout,
                  toolbox.attr_activation, toolbox.attr_optimizer))


# Popülasyon oluşturma
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
import pickle

# Register evaluation function
toolbox.register("evaluate", evaluate_ga)

# Initialize the population
population = toolbox.population(n=50)

# Run the GA for the first 50 generations
result = algorithms.eaSimple(population, toolbox, cxpb=0.3, mutpb=0.05, ngen=50, verbose=True)

# Save progress
with open('ga_progress.pkl', 'wb') as f:
    pickle.dump(population, f)

# Load the saved population
with open('ga_progress.pkl', 'rb') as f:
    population = pickle.load(f)

# Continue the GA for an additional 25 generations
result = algorithms.eaSimple(population, toolbox, cxpb=0.3, mutpb=0.05, ngen=25, verbose=True)

# Get the best individual after all generations
best_individual = tools.selBest(population, k=1)[0]
print("En iyi birey (GA):", best_individual)
print("Doğruluk (GA):", evaluate_ga(best_individual))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


gen	nevals
0  	50    
1  	21    
2  	18    
3  	23    
4  	12    
5  	17    
6  	22    
7  	13    
8  	17    
9  	15    
10 	24    
11 	4     
12 	13    
13 	20    
14 	19    
15 	18    
16 	18    
17 	25    
18 	7     
19 	26    
20 	18    
21 	22    
22 	13    
23 	7     
24 	17    
25 	16    
26 	18    
27 	12    
28 	23    
29 	14    
30 	17    
31 	26    
32 	23    
33 	20    
34 	10    
35 	15    
36 	15    
37 	11    
38 	18    
39 	20    
40 	18    
41 	19    
42 	15    
43 	10    
44 	24    
45 	17    
46 	20    
47 	8     
48 	20    
49 	18    
50 	11    
gen	nevals
0  	0     
1  	14    
2  	17    
3  	18    
4  	16    
5  	21    
6  	16    
7  	17    
8  	12    
9  	12    
10 	29    


In [ ]:
def pso_model(solution):
    learning_rate, num_layers, num_neurons, activation_index = solution
    activation_functions = ['relu', 'tanh', 'sigmoid']
    activation = activation_functions[int(activation_index)]
    model = build_model(learning_rate, int(num_layers), int(num_neurons), activation)
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return 1 - accuracy  # Minimizasyon için 1-accuracy döndürülüyor

problem_dict = {
    "fit_func": pso_model,
    "lb": [0.0001, 1, 16, 0],
    "ub": [0.1, 5, 128, 2],
    "minmax": "min",
    "log_to": None,
}

In [ ]:
model = BasePSO(problem_dict, epoch=1000, pop_size=50)
best_position, best_fitness = model.train()
print("En iyi birey (PSO):", best_position)
print("Doğruluk (PSO):", 1 - best_fitness)